In [ ]:
import kagglehub
import os
import re
from collections import Counter

## Extract Data from Kagglehub

In [22]:
path = kagglehub.dataset_download("wcukierski/enron-email-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/elohookoloko/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2


## View files in directory

In [23]:

dataset_path = "/Users/elohookoloko/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2"

for root, dirs, files in os.walk(dataset_path):
    print(root, dirs, files)
    break   # only print top-level


/Users/elohookoloko/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2 [] ['emails.csv']


## Directory contains a neatly organized csv files with the emails so read into a pandas dataframe

In [24]:
import pandas as pd

file_path = "/Users/elohookoloko/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2/emails.csv"

df = pd.read_csv(file_path)
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


## Explore the dataset

In [25]:
df.columns

Index(['file', 'message'], dtype='object')

In [26]:
df.shape

(517401, 2)

In [27]:
print(df['message'].iloc[0])
print(df['message'].iloc[1])
print(df['message'].iloc[2])
print(df['message'].iloc[3])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 
Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have

In [28]:
print(df['message'].iloc[100])

Message-ID: <15940494.1075855689309.JavaMail.evans@thyme>
Date: Wed, 16 Aug 2000 05:35:00 -0700 (PDT)
From: phillip.allen@enron.com
To: cooper.richey@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Cooper Richey
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

I tried the new address but I don't have access.  also, what do I need to 
enter under domain?


In [29]:
print(df['message'].iloc[480])

Message-ID: <33141543.1075855727805.JavaMail.evans@thyme>
Date: Thu, 1 Feb 2001 03:07:00 -0800 (PST)
From: phillip.allen@enron.com
To: jeff.richter@enron.com
Subject: Re: web site
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Jeff Richter
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_June2001\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Nice.  how much?

Are you trying to keep the economy going?


## Prep the dataset to be used

extract the potentially useful fields

In [30]:
def extract_subject(message):
    match = re.search(r"Subject:(.*)", message)
    return match.group(1).strip() if match else None

df["subject"] = df["message"].apply(extract_subject)

In [31]:
def extract_body(message):
    parts = message.split("\n\n", 1)
    return parts[1].strip() if len(parts) > 1 else ""
    
df["body"] = df["message"].apply(extract_body)

In [32]:
df[["subject", "body"]].head()


,subject,body
0,,Here is our forecast
1,Re:,Traveling to have a business meeting takes the...
2,Re: test,test successful. way to go!!!
3,,"Randy,\n\n Can you send me a schedule of the s..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.


In [33]:
df[["subject", "body"]].iloc[0]


subject                        
body       Here is our forecast
Name: 0, dtype: object

In [34]:
df[["subject", "body"]].iloc[100]

subject                                                  Re:
body       I tried the new address but I don't have acces...
Name: 100, dtype: object

In [35]:
df[["subject", "body"]].tail()

,subject,body
517396,Trade with John Lavorato,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,Gas Hedges,Some of my position is with the Alberta Term b...
517398,RE: CONFIDENTIAL,2\n\n -----Original Message-----\nFrom: \tDouc...
517399,Calgary Analyst/Associate,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...
517400,RE: ali's essays,i think the YMCA has a class that is for peopl...


## I wanted to lightly clean the dataset not taking out too much information that could be an indecator or spam/scam

In [37]:
def clean_body(text):
    text = re.sub(r"(?i)forwarded by.*", "", text)
    text = re.sub(r"http\S+", "", text)      # remove URLs
    text = re.sub(r"\s+", " ", text)         # collapse whitespace
    return text.strip()

df["clean_body"] = df["body"].apply(clean_body)


## I want to Tokenize my cleaaned dataframe

In [38]:
def tokenize(text):
    if not isinstance(text, str):
        return []
    text = text.lower()
    # keep sequences of letters and apostrophes (e.g. don't, can't)
    tokens = re.findall(r"[a-z']+", text)
    return tokens

df["tokens"] = df["clean_body"].apply(tokenize)


## Building a word frequency dictionary(tf-idf kind of)

In [40]:
word_counts = Counter()

for tokens in df["tokens"]:
    word_counts.update(tokens)

In [41]:
print("Vocabulary size:", len(word_counts)) # count of all unique words in this dataset

Vocabulary size: 595968


In [42]:
df.tail()

,file,message,subject,body,clean_body,tokens
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,Trade with John Lavorato,This is a trade with OIL-SPEC-HEDGE-NG (John L...,This is a trade with OIL-SPEC-HEDGE-NG (John L...,"[this, is, a, trade, with, oil, spec, hedge, n..."
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...,Gas Hedges,Some of my position is with the Alberta Term b...,Some of my position is with the Alberta Term b...,"[some, of, my, position, is, with, the, albert..."
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...,RE: CONFIDENTIAL,2\n\n -----Original Message-----\nFrom: \tDouc...,"2 -----Original Message----- From: Doucet, Daw...","[original, message, from, doucet, dawn, sent, ..."
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...,Calgary Analyst/Associate,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,Analyst Rank Stephane Brodeur 1 Chad Clark 1 I...,"[analyst, rank, stephane, brodeur, chad, clark..."
517400,zufferli-j/sent_items/99.,Message-ID: <28618979.1075842030037.JavaMail.e...,RE: ali's essays,i think the YMCA has a class that is for peopl...,i think the YMCA has a class that is for peopl...,"[i, think, the, ymca, has, a, class, that, is,..."


In [46]:
df[["subject", "tokens"]].tail(20)

,subject,tokens
517381,FW: I:\NewCo file inventory,"[original, message, from, mckeel, richard, sen..."
517382,RE:,"[nothing, new, here, when, is, your, flight, h..."
517383,RE: NETCO,"[in, addition, to, a, list, of, software, and,..."
517384,Digital Certificate for TAU,"[please, set, up, access, for, the, digital, c..."
517385,FW: Generation Maintenance Planning and Relate...,"[original, message, from, ritter, fred, fred, ..."
517386,FW: hello,"[original, message, from, john, berger, wberge..."
517387,FW: Strictly Confidential,"[original, message, from, milnthorp, rob, sent..."
517388,FW: Strictly Confidential,"[here, is, the, info, for, the, commercial, st..."
517389,RE: Supply Shortfall Procedures,"[enron, is, willing, to, perform, the, operati..."
517390,RE: EOL - Multiple Trading Books (URGENT),"[i, will, have, books, cand, mgmt, bas, for, a..."


## Create a dataframe for the words in the dictionary for easier inspection

In [ ]:
vocab_df = pd.DataFrame(word_counts.items(), columns=["word", "count"])
vocab_df = vocab_df.sort_values("count", ascending=False).reset_index(drop=True)

vocab_df.tail(20)

,word,count
0,the,5657510
1,to,4033794
2,and,2564324
3,of,2361328
4,a,2097707
5,in,1746310
6,enron,1727554
7,for,1444776
8,is,1219569
9,you,1143460


In [50]:
vocab_df.head(50)

,word,count
0,the,5657510
1,to,4033794
2,and,2564324
3,of,2361328
4,a,2097707
5,in,1746310
6,enron,1727554
7,for,1444776
8,is,1219569
9,you,1143460


## Remove stop words

## Dataframe with no stopwords

In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elohookoloko/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

vocab_no_stop = vocab_df[~vocab_df["word"].isin(stop_words)].copy()


In [53]:
vocab_no_stop.head(30)

,word,count
6,enron,1727554
12,ect,1058161
14,com,987921
28,hou,499467
32,subject,446793
33,e,437381
34,pm,391278
35,please,388591
39,would,330470
44,power,309199


## To explore medium frequency words as the most friequent ones still kind of have little meaning

In [54]:
medium_freq = vocab_no_stop[(vocab_no_stop["count"] >= 20) & (vocab_no_stop["count"] <= 500)]

In [55]:
medium_freq.head(50)

,word,count
12794,eighth,500
12795,hc,500
12796,almeida,500
12797,lung,500
12798,looming,500
12799,musical,500
12800,sonnier,500
12801,blues,500
12802,himendra,500
12803,enhancing,500


## Explore dataset for suspicious words

In [56]:
mask = vocab_no_stop["word"].str.contains(
    "wire|pay|money|transfer|payment|account|bank|invoice|credit|debit|loan|secret|confidential|urgent|asap|password|login|access",
    case=False,
    regex=True
)

vocab_suspicious_like = vocab_no_stop[mask].sort_values("count", ascending=False)



In [57]:
vocab_suspicious_like.head(50)

,word,count
176,credit,84050
248,access,64867
377,pay,47223
438,money,39920
515,confidential,34507
550,account,32455
688,bankruptcy,26380
697,bank,25886
718,payment,25155
944,accounting,19260


make sure to modify this list with words that we find in the above  lists

In [58]:
suspicious_keywords = [
    # money / transactions
    "wire", "wiring", "transfer", "transfers", "payment", "payments",
    "invoice", "invoices", "billing", "rebate", "refund",
    "account", "accounts", "bank", "credit", "debit", "loan",
    # secrecy / sensitivity
    "confidential", "secret", "strictly", "offshore", "undisclosed",
    # urgency
    "urgent", "immediately", "asap", "deadline",
    # access / security
    "password", "login", "logon", "credentials", "access", "authorize",
]


In [60]:
suspicious_keywords = [
    # money / transactions
    "wire", "transfer",  "payment", "payments",
    "invoice", "billing", "rebate", "refund",
    "account", "accounts", "bank",
    # secrecy / sensitivity
    "confidential", "secret", "strictly", "offshore", "undisclosed",
    # urgency
    "urgent", "immediately", "asap", "deadline",
    # access / security
    "password", "login", "credentials", "access", "authorize",
]

# 2. Helper functions
def contains_suspicious_keyword(text, keywords):
    if not isinstance(text, str):
        return False
    text = text.lower()
    return any(kw in text for kw in keywords)

def find_keyword(text, keywords):
    if not isinstance(text, str):
        return None
    text = text.lower()
    for kw in keywords:
        if kw in text:
            return kw
    return None

# 3. Filter emails that contain at least one suspicious keyword
mask = df["clean_body"].apply(lambda x: contains_suspicious_keyword(x, suspicious_keywords))
suspicious_emails_df = df.loc[mask].copy()

# 4. Add a column showing which keyword triggered the match
suspicious_emails_df["trigger_word"] = suspicious_emails_df["clean_body"].apply(
    lambda x: find_keyword(x, suspicious_keywords)
)

# 5. Show full email bodies (no truncation) for visual inspection
pd.set_option("display.max_colwidth", None)

# 6. Display the first 20 suspicious emails (adjust number as you like)
suspicious_emails_df[["subject", "trigger_word", "clean_body"]].head(20)


subject  \
8                                      Re: High Speed Internet Access   
9             FW: fixed forward or other Collar floor gas price terms   
17                                                                      
20                                                                      
22                                    Re: Original Sept check/closing   
25                                               Investment Structure   
26                                               Investment Structure   
37                                                                      
40                                                                      
46                                                 Re: Sept 1 Payment   
47                                                                      
48                                        Contact list for mid market   
50                                      Re: 2- SURVEY - PHILLIP ALLEN   
55                                                  Re: Sagewood etc.   
61                              4-URGENT - OWA Please print this now.   
63                                                                Wow   
66  RE: Receipt of Team Selection Form - Executive Impact & Influence   
70                                                                      
71      Receipt of Team Selection Form - Executive Impact & Influence   
72                                        2- SURVEY/INFORMATION EMAIL   

   trigger_word  \
8         login   
9       account   
17     password   
20      payment   
22      payment   
25      payment   
26      payment   
37     password   
40      payment   
46      payment   
47         bank   
48      account   
50        login   
55      payment   
61       urgent   
63      account   
66     transfer   
70       access   
71     transfer   
72        login   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [61]:
suspicious_emails_df[["subject", "trigger_word", "clean_body"]].tail(20)

,subject,trigger_word,clean_body
517349,RE:,payment,"I am reading your mail, I have asked for the same thing regarding my base. Especially since I came up here with a base salary converted at 1.48 CDN/USD and with no raise since being here at 1.61 CDN/USD I have received a 9% pay cut relative to my peers in Houston. Not to mention that my base salary is $50-$100 USD/year lower than what other shops are offering. However, after three attempts, I have been unsuccessful in championing my own cause. -----Original Message----- From: Lambie, Chris Sent: Friday, January 18, 2002 1:51 PM To: Zufferli, John Subject: Zuff Just a quick note after my first look at the numbers. The only concern I have is with the base number. Seeing as there is no money up front, the future payments look a lot more like long term comp. than any real signing or retention bonus. When I came to Enron two years ago I took a $35,000 decrease in base pay in exchange for options and the opportunity to trade for number 1. That is no longer the case (no options and UBS is not #1) therefore I would look for my base to better reflect market on a go forward basis. If you are able to accommodate a reasonable adjustment in base I would be very excited about the opportunity to sign with UBS. I would like to talk about this on Monday, after I have had a chance to look at the paper in full. Chris."
517354,RE: Ontario Update,confidential,"do you have the internet addresses and directory details put togethor yet -----Original Message----- From: Tripp, Garrett Sent: Thursday, January 17, 2002 12:32 PM To: Milnthorp, Rob Cc: Zufferli, John Subject: RE: Ontario Update Rob, Markets My marks show how we believe the market would trade if the market were open today, considering neighboring markets, transmission/wheeling costs, and fuel costs. More of a expected liquidation value vs. the current Ontario market players bids and offers. The market on 3yr 7x24 is being shown by a broker: $46-$48, and on the wholesale side I doubt the bid is actually there, or if so it is only a piece deep (50MW). As a result, I would show an actually two way of $43-$47.50 and be coming from the sell side to the industrial customers who may pay in the $46-$47 range. Peak, my 3yr mark is $52, but given the often bullish nature of new markets, I would be a bid for this summer and then look to offer the 3yr in the low fifties. Upcoming changes in the Ontario system (Nodal vs. Zonal pricing) would have an effect on these term deals. Staffing and Services Trading On the trading side, I will need to rehire personnel for the roles we laid off (an analyst and a temp IT programmer for fundies development). Analyst wise, I am interested in bringing Brian Loy back if he is still available. He has a good knowledge of the NUG contracts and is a good fit for a trading desk. As our need for providing IMO Energy and Operating Reserve Services (bidding for specific NUGS and Dispatchable Loads) grows, it may be a good idea to bring Heather Mcillmurray over from the settlements group. I have had initial discussions with Richard Crouch, Paul Devries and Heather on this matter. Origination This is your area of expertise. I would suggest two originators and an analyst. If Dan Dorland is staying, he could continue on the origination side providing education to our new origination team, and being the analyst to assist the OEFC-NUG point personnel (Brian Loy's previous role). IT Implementing the trading and VMS systems is a very important issue where timing will become an issue. I would suggest a short term (6mo) contract with Mike Marryott to manage the system's side of this process and to integrate with the Estate and NETCO on trading and settlements systems (specifically David Posten's VMS team). His location should be in Toronto. Let's get on these issues as soon as possible. On specific personnel, I would have liked to have retained Jeff Borg and Chris Gaffney. Office Space Tembec, our neighbor on this floor, has often ind

The approach might need to be changed because after manually viewing the emails that contain these trigger words for potentially fraudulent emails, it still doesnt look like any of them are fraudulent

before i comepletely switch directions, i want to try one last thing, most fraudulent emails come from external messages so i want to review only external messages with these suspisiouc keywards

In [62]:

def extract_from(text):
    if not isinstance(text, str):
        return None
    match = re.search(r"From:\s*([^\s]+)", text)    # captures the email address after "From:"
    return match.group(1).lower() if match else None

df["from_email"] = df["message"].apply(extract_from)

In [63]:
df["from_email"].head()


0    phillip.allen@enron.com
1    phillip.allen@enron.com
2    phillip.allen@enron.com
3    phillip.allen@enron.com
4    phillip.allen@enron.com
Name: from_email, dtype: object

In [64]:
df["is_internal"] = df["from_email"].apply(
    lambda x: x.endswith("enron.com") if isinstance(x, str) else False
)


In [65]:
external_df = df[df["is_internal"] == False].copy()


In [66]:
external_df[["from_email", "subject", "clean_body"]].head(20)


from_email  \
602  1.11913372.-2@multexinvestornetwork.com   
603              messenger@ecm.bloomberg.com   
614                         aod@newsdata.com   
656       market-reply@listserv.dowjones.com   
678                   webmaster@earnings.com   
700                 yild@zdemail.zdlists.com   
711     bounce-news-932653@lists.autoweb.com   
712                         aod@newsdata.com   
813                         bobregon@bga.com   
820      subscriptions@intelligencepress.com   
838              rob_tom@freenet.carleton.ca   
839                            calxa@aol.com   
843                   ei_editor@ftenergy.com   
846                   billc@greenbuilder.com   
871                      matt@fastpacket.net   
872                         jfreeman@ssm.net   
873                owner-strawbale@crest.org   
876                       grensheltr@aol.com   
880             yahoo-delivers@yahoo-inc.com   
881                   ei_editor@ftenergy.com   

                                                                subject  \
602  December 14, 2000 - Bear Stearns' predictions for telecom in Latin   
603                                        Bloomberg Power Lines Report   
614                                 Special report coming from NewsData   
656                       MARKET ALERT: Nasdaq Composite Ends Down 3.7%   
678                              FS Van Kasper Initiates Coverage of NT   
700   Y-Life Daily: Bush will almost definitely be prez / Coach K chats   
711   December Newsletter - Factory Incentives are at a year-long high!   
712                                     Western Price Survey 12/13/2000   
813                                         Central Texas Bale Resource   
820                       NGI Publications - Thursday, 14 December 2000   
838                                     Re: History of  Lime and Cement   
839                                         History of  Lime and Cement   
843   Demand-side management garnering more attention. Deregulation spa   
846                                                      Re: Newsgroups   
871                                                  RE: concrete stain   
872                                        The 1999 Hemp Year in Review   
873                                                                       
876                                                  Re: concrete stain   
880                                         Yahoo! Newsletter, May 2001   
881                          Texas puts reliability rules through paces   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [67]:
external_suspicious_df = external_df[
    external_df["clean_body"].apply(lambda x: contains_suspicious_keyword(x, suspicious_keywords))
].copy()

# 7. Add which keyword triggered
external_suspicious_df["trigger_word"] = external_suspicious_df["clean_body"].apply(
    lambda x: find_keyword(x, suspicious_keywords)
)

# 8. Show FULL text (no truncation)
pd.set_option("display.max_colwidth", None)

# 9. Display the first 20 suspicious external emails
external_suspicious_df[["from_email", "subject", "trigger_word", "clean_body"]].head(20)

from_email  \
602   1.11913372.-2@multexinvestornetwork.com   
678                    webmaster@earnings.com   
711      bounce-news-932653@lists.autoweb.com   
820       subscriptions@intelligencepress.com   
843                    ei_editor@ftenergy.com   
880              yahoo-delivers@yahoo-inc.com   
881                    ei_editor@ftenergy.com   
883                    webmaster@earnings.com   
884                announce@inbox.nytimes.com   
886                       jsmith@austintx.com   
1232                     pallen70@hotmail.com   
1234                   webmaster@earnings.com   
1242               mery.l.brown@accenture.com   
1244                     ei_editor@platts.com   
1245  anchordesk_daily@anchordesk.zdlists.com   
1246      subscriptions@intelligencepress.com   
1248                   webmaster@earnings.com   
1249             yahoo-delivers@yahoo-inc.com   
1253         edelivery@salomonsmithbarney.com   
1255                     showtimes@amazon.com   

                                                                 subject  \
602   December 14, 2000 - Bear Stearns' predictions for telecom in Latin   
678                               FS Van Kasper Initiates Coverage of NT   
711    December Newsletter - Factory Incentives are at a year-long high!   
820                        NGI Publications - Thursday, 14 December 2000   
843    Demand-side management garnering more attention. Deregulation spa   
880                                          Yahoo! Newsletter, May 2001   
881                           Texas puts reliability rules through paces   
883                        Freidman, Billings Initiates Coverage of PMCS   
884                          Pre-selected NextCard Visa! As low as 2.99%   
886                                                      RE: stage coach   
1232                                                 Fwd: Bishops Corner   
1234                                             NT Earnings Information   
1242                                                           Thank You   
1244           A hard act to follow: the future for energy without Enron   
1245                                    Meet the dark side of Windows XP   
1246                        NGI Publications - Monday, October 22nd 2001   
1248          Reminder: AXP Q3 Earnings Announcement on October 22, 2001   
1249                                     Yahoo! Newsletter, October 2001   
1253                                  E-delivery Notification - Confirms   
1255                         Your Weekly Movie Showtimes from Amazon.com   

      trigger_word  \
602         refund   
678        account   
711         rebate   
820       password   
843         access   
880        account   
881         refund   
883        billing   
884       transfer   
886           asap   
1232       account   
1234          wire   
1242  confidential   
1244      transfer   
1245      password   
1246      password   
1248          wire   
1249       account   
1253      password   
1255       account   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 